# Información y Programación Financiera, TI, Algoritmos y Ciencia de Datos

Neftalí Valdez

<a href="http://twitter.com/neftalivldz" target="_blank">@neftalivldz</a> | <a href="mailto:nvaldez@tec.mx">nvaldez@tec.mx</a>

In [30]:
#%pip install statsmodels

In [52]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "../../Configuration"

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [32]:
import lseg.data as ld
from lseg.data.discovery import Peers

import pandas as pd
import numpy as np
import math
from statsmodels.api import OLS, add_constant
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import Box, Layout
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")


In [33]:
ld.open_session()

<lseg.data.session.Definition object at 0x303362350 {name='workspace'}>

In [34]:
ric = widgets.Text(
    value='',
    placeholder='Type something',
    description='RIC:',
    disabled=False
)

box_layout = Layout(display='flex',
                    flex_flow='row wrap',
                    align_items='stretch',
                    border='solid',
                    width='100%')

items = [ric]
box = Box(children=items,layout=box_layout)

box

Box(children=(Text(value='', description='RIC:', placeholder='Type something'),), layout=Layout(align_items='s…

In [35]:
print(ric.value)


RHMG.DE


In [36]:
peers = Peers(ric.value)
df = ld.get_data([ric.value, *peers],['TR.RIC','TR.CommonName',
                                                       'TR.HQCountryCode','TR.WACC','TR.WACCERP',
                                                       'TR.WACCDebtWeight','TR.WACCEquityWeight',
                                                       'TR.WACCBeta','TR.WACCCompanyTaxRateMedian'])
df

,Instrument,RIC,Company Common Name,Country ISO Code of Headquarters,"Weighted Average Cost of Capital, (%)","WACC Equity Risk Premium, (%)","WACC Debt Weight, (%)","WACC Equity Weight, (%)",Beta,"WACC Company Tax Rate Median, (%)"
0,RHMG.DE,RHMG.DE,Rheinmetall AG,DE,5.603637,4.11148,8.299574,91.700426,0.454701,25.48747
1,MTXGn.DE,MTXGn.DE,MTU Aero Engines AG,DE,7.965887,4.11148,0.0,100.0,0.966281,26.66667
2,HAGG.F,HAGG.F,Hensoldt AG,DE,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,R3NK.D,R3NK.D,RENK Group AG,DE,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,DUEG.DE,DUEG.DE,Duerr AG,DE,10.290985,4.11148,0.0,100.0,1.531794,35.64818
5,RAAG.DE,RAAG.DE,Rational AG,DE,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,TCFP.PA,TCFP.PA,Thales SA,FR,6.367531,4.258498,21.218482,78.781518,0.786968,16.71121
7,BAES.L,BAES.L,BAE Systems PLC,GB,5.861027,5.920283,22.866102,77.133898,0.361872,14.09774
8,AIR.PA,AIR.PA,Airbus SE,NL,9.6372,4.615838,12.885061,87.114939,1.429708,21.37117
9,G1AG.DE,G1AG.DE,GEA Group AG,DE,7.553699,4.11148,3.596319,96.403681,0.911182,18.85489


In [37]:
peers = ['MTXGn.DE', 'DUEG.DE', 'TCFP.PA', 'BAES.L']


## Razones Financieras

Deuda - Apalancamiento Razones de eficiencia

In [39]:
RICS = ['RHMG.DE', 'MTXGn.DE', 'DUEG.DE', 'TCFP.PA', 'BAES.L']
fields = ['TR.EpsSmartEst','TR.EPSMean','TR.EpsPreSurprisePct','TR.EpsPreSurprise','TR.EpsPreSurpriseFlag','TR.EPSMedian','TR.EPSLow','TR.EPSHigh']

eps = ld.get_data(RICS, fields)
eps

,Instrument,Earnings Per Share - SmartEstimate®,Earnings Per Share - Mean,Earnings Per Share - Predicted Surprise PCT,Earnings Per Share - Predicted Surprise,Earnings Per Share - Predicted Surprise Flag,Earnings Per Share - Median,Earnings Per Share - Low,Earnings Per Share - High
0,RHMG.DE,29.71275,29.53824,0.591,0.17451,0,29.74,26.5,33.01
1,MTXGn.DE,16.38662,16.09142,1.835,0.2952,0,16.32,14.402,17.23
2,DUEG.DE,2.44289,2.46743,-0.995,-0.02454,0,2.36,2.13,2.99
3,TCFP.PA,9.82668,9.52663,3.15,0.30005,0,9.5345,7.62,11.42
4,BAES.L,0.758148,0.752448,0.758,0.0057,0,0.7563,0.67,0.82436


In [40]:
ts = ld.get_data(RICS,
                   ['TR.TotalRevenue(SDate=2010-01-01,EDate=2024-12-15,Frq=Y).date',
                    'TR.TotalRevenue(SDate=2010-01-02,EDate=2024-12-15,Frq=Y)',
                    'TR.EBITDAMarginPercent(SDate=2010-01-02,EDate=2024-12-15,Frq=Y)',
                    'TR.EBITMarginPercent(SDate=2010-01-02,EDate=2024-12-15,Frq=Y)',
                    'TR.NetProfitMargin(SDate=2010-01-02,EDate=2024-12-15,Frq=Y)',
                    'TR.CapitalExpendituresCFStmt(SDate=2010-01-02,EDate=2024-12-15,Frq=Y)',
                    'TR.PriceClose(SDate=2010-01-02,EDate=2025-03-12,Frq=Y)'
                   ])
ts2 = ts.drop_duplicates()
ts2

,Instrument,Date,Total Revenue,"EBITDA Margin, Percent","EBIT Margin, Percent","Net Profit Margin, (%)","Capital Expenditures, Discrete",Price Close
0,RHMG.DE,2008-12-31,4014000000,10.413553,6.30294,3.51271,200000000,44.74
1,RHMG.DE,2009-12-31,3547000000,9.641951,5.018325,-1.63518,145000000,60.17
2,RHMG.DE,2010-12-31,4125000000,11.587879,7.587879,3.92727,186000000,35.795
3,RHMG.DE,2011-12-31,4454000000,11.854513,7.723395,4.78222,197000000,37.3
4,RHMG.DE,2012-12-31,4704000000,10.352891,6.25,3.67772,234000000,43.62
...,...,...,...,...,...,...,...,...
75,BAES.L,2019-12-31,18305000000,13.187654,9.855231,8.06337,470000000,488.8
76,BAES.L,2020-12-31,19277000000,12.714634,9.25974,6.7386,477000000,549.8
77,BAES.L,2021-12-31,19521000000,11.505558,7.904308,9.00569,462000000,856.0
78,BAES.L,2022-12-31,21258000000,13.538433,9.9539,7.48424,693000000,1116.0


In [45]:
import datetime as dt
ts2['CapEx_TR'] = ts2['Capital Expenditures, Discrete']*100/ts2['Total Revenue']
ts2['Total Revenue lag'] = ts2.groupby(["Instrument"])['Total Revenue'].shift(1)
ts2['TotalRevenueGrowth'] = ((ts2['Total Revenue']/ts2['Total Revenue lag'])-1)*100
ts2['Year'] = ts2['Date'].dt.year
ts2 = ts2.dropna()

In [48]:
ts3 = ts2.loc[ts2['Instrument'] == ric.value][['Year', 'EBITDA Margin, Percent', 'EBIT Margin, Percent', 'Net Profit Margin, (%)', 'CapEx_TR', 'TotalRevenueGrowth', 'Price Close']]
ts3

,Year,"EBITDA Margin, Percent","EBIT Margin, Percent","Net Profit Margin, (%)",CapEx_TR,TotalRevenueGrowth,Price Close
1,2009.0,9.641951,5.018325,-1.63518,4.087962,-11.63428,60.17
2,2010.0,11.587879,7.587879,3.92727,4.509091,16.295461,35.795
3,2011.0,11.854513,7.723395,4.78222,4.422991,7.975758,37.3
4,2012.0,10.352891,6.25,3.67772,4.97449,5.612932,43.62
5,2013.0,9.486077,4.641159,0.88295,4.324202,-6.10119,35.855
6,2014.0,7.188567,2.943686,0.2773,6.05802,6.135386,61.48
7,2015.0,9.261046,5.363689,2.91337,5.981092,10.558874,64.5
8,2016.0,10.46055,6.586933,3.57015,5.051767,8.084121,106.1
9,2017.0,10.634328,6.902985,3.79919,4.579376,5.248126,80.08
10,2018.0,12.329213,7.856213,4.96096,4.50553,4.274084,103.55


In [49]:
ts_noMC = ts2.loc[ts2['Instrument'] != ric.value]
ts_noMC

,Instrument,Date,Total Revenue,"EBITDA Margin, Percent","EBIT Margin, Percent","Net Profit Margin, (%)","Capital Expenditures, Discrete",Price Close,CapEx_TR,Total Revenue lag,TotalRevenueGrowth,Year
17,MTXGn.DE,2009-12-31,2610800000,14.14892,9.307492,5.40064,140300000,50.61,5.373832,2724300000,-4.166208,2009.0
18,MTXGn.DE,2010-12-31,2707400000,14.74108,9.906183,5.25227,109400000,50.0,4.040777,2610800000,3.700015,2010.0
19,MTXGn.DE,2011-12-31,2932100000,15.664541,11.04669,5.42956,160400000,70.74,5.470482,2707400000,8.299476,2011.0
20,MTXGn.DE,2012-12-31,3378600000,16.281892,11.839223,5.17374,364800000,71.0,10.797372,2932100000,15.227994,2012.0
21,MTXGn.DE,2013-12-31,3574100000,14.761758,10.153605,4.65292,179100000,71.86,5.011052,3378600000,5.78642,2013.0
22,MTXGn.DE,2014-12-31,3913900000,13.523595,9.486701,4.99757,201500000,90.1,5.148318,3574100000,9.507289,2014.0
23,MTXGn.DE,2015-12-31,4435300000,13.791626,10.012851,4.90609,272700000,110.7,6.1484,3913900000,13.321751,2015.0
24,MTXGn.DE,2016-12-31,4732700000,13.36869,9.618188,6.59666,267500000,149.3,5.652165,4435300000,6.705296,2016.0
25,MTXGn.DE,2017-12-31,3897400000,14.945861,11.194643,9.1112,305000000,158.8,7.82573,4732700000,-17.649545,2017.0
26,MTXGn.DE,2018-12-31,4567100000,15.817477,12.500274,9.78739,319800000,258.1,7.002255,3897400000,17.18325,2018.0


In [50]:
mean_grouped = ts_noMC.groupby('Year')[['EBITDA Margin, Percent', 'EBIT Margin, Percent', 'Net Profit Margin, (%)', 'CapEx_TR', 'TotalRevenueGrowth', 'Price Close']].mean().reset_index()
mean_grouped_renamed = mean_grouped.rename(columns={'EBITDA Margin, Percent': 'EBITDA Margin, Percent peers', 
                   'EBIT Margin, Percent': 'EBIT Margin, Percent peers', 
                   "Net Profit Margin, (%)": "Net Profit Margin, (%) peers",
                   'CapEx_TR': 'CapEx_TR peers',
                   'TotalRevenueGrowth': 'TotalRevenueGrowth peers',
                    'Price Close': 'Price Close peers'
                  }, errors="raise")
mean_grouped_renamed 

,Year,"EBITDA Margin, Percent peers","EBIT Margin, Percent peers","Net Profit Margin, (%) peers",CapEx_TR peers,TotalRevenueGrowth peers,Price Close peers
0,2009.0,8.412959,5.156649,0.233032,3.204057,-3.251867,103.190625
1,2010.0,8.811857,5.442571,2.379723,2.412574,6.403537,92.049688
2,2011.0,11.263259,8.213299,4.89412,2.668614,11.159148,116.16375
3,2012.0,11.574593,8.5035,4.863915,4.401139,10.672236,145.11125
4,2013.0,11.614608,8.724959,3.996117,2.937012,-0.799816,155.77875
5,2014.0,11.538279,8.467978,5.27948,3.263981,2.539758,173.9
6,2015.0,10.878013,8.013907,5.025942,3.655826,19.203774,208.21875
7,2016.0,11.156367,8.296316,5.79411,3.479304,3.345932,217.13375
8,2017.0,11.757726,8.965056,5.80646,3.957286,-3.655078,189.1325
9,2018.0,12.400207,9.750953,6.48838,3.573476,5.795142,239.0025


In [53]:
ts_vs_peers = ts3.merge(mean_grouped_renamed, how='left', on='Year')
ts_vs_peers

,Year,"EBITDA Margin, Percent","EBIT Margin, Percent","Net Profit Margin, (%)",CapEx_TR,TotalRevenueGrowth,Price Close,"EBITDA Margin, Percent peers","EBIT Margin, Percent peers","Net Profit Margin, (%) peers",CapEx_TR peers,TotalRevenueGrowth peers,Price Close peers
0,2009.0,9.641951,5.018325,-1.63518,4.087962,-11.63428,60.17,8.412959,5.156649,0.233032,3.204057,-3.251867,103.190625
1,2010.0,11.587879,7.587879,3.92727,4.509091,16.295461,35.795,8.811857,5.442571,2.379723,2.412574,6.403537,92.049688
2,2011.0,11.854513,7.723395,4.78222,4.422991,7.975758,37.3,11.263259,8.213299,4.89412,2.668614,11.159148,116.16375
3,2012.0,10.352891,6.25,3.67772,4.97449,5.612932,43.62,11.574593,8.5035,4.863915,4.401139,10.672236,145.11125
4,2013.0,9.486077,4.641159,0.88295,4.324202,-6.10119,35.855,11.614608,8.724959,3.996117,2.937012,-0.799816,155.77875
5,2014.0,7.188567,2.943686,0.2773,6.05802,6.135386,61.48,11.538279,8.467978,5.27948,3.263981,2.539758,173.9
6,2015.0,9.261046,5.363689,2.91337,5.981092,10.558874,64.5,10.878013,8.013907,5.025942,3.655826,19.203774,208.21875
7,2016.0,10.46055,6.586933,3.57015,5.051767,8.084121,106.1,11.156367,8.296316,5.79411,3.479304,3.345932,217.13375
8,2017.0,10.634328,6.902985,3.79919,4.579376,5.248126,80.08,11.757726,8.965056,5.80646,3.957286,-3.655078,189.1325
9,2018.0,12.329213,7.856213,4.96096,4.50553,4.274084,103.55,12.400207,9.750953,6.48838,3.573476,5.795142,239.0025


In [54]:
ts_vs_peers['EBITDA'] = ts_vs_peers['EBITDA Margin, Percent'] > (ts_vs_peers['EBITDA Margin, Percent peers']*1.5)
ts_vs_peers['EBIT'] = ts_vs_peers['EBIT Margin, Percent'] > (ts_vs_peers['EBIT Margin, Percent peers']*1.5)
ts_vs_peers['Profit'] = ts_vs_peers['Net Profit Margin, (%)'] > (ts_vs_peers['Net Profit Margin, (%) peers']*1.5)
ts_vs_peers['Capex'] = ts_vs_peers['CapEx_TR'] > (ts_vs_peers['CapEx_TR peers']*1.2)
ts_vs_peers['Revenue'] = ts_vs_peers['TotalRevenueGrowth'] > (ts_vs_peers['TotalRevenueGrowth peers']*1.2)
ts_vs_peers['tool_sum'] = ts_vs_peers[['EBITDA', 'EBIT', 'Profit', 'Capex', 'Revenue']].sum(1)

ts_vs_peers

,Year,"EBITDA Margin, Percent","EBIT Margin, Percent","Net Profit Margin, (%)",CapEx_TR,TotalRevenueGrowth,Price Close,"EBITDA Margin, Percent peers","EBIT Margin, Percent peers","Net Profit Margin, (%) peers",CapEx_TR peers,TotalRevenueGrowth peers,Price Close peers,EBITDA,EBIT,Profit,Capex,Revenue,tool_sum
0,2009.0,9.641951,5.018325,-1.63518,4.087962,-11.63428,60.17,8.412959,5.156649,0.233032,3.204057,-3.251867,103.190625,False,False,False,True,False,1
1,2010.0,11.587879,7.587879,3.92727,4.509091,16.295461,35.795,8.811857,5.442571,2.379723,2.412574,6.403537,92.049688,False,False,True,True,True,3
2,2011.0,11.854513,7.723395,4.78222,4.422991,7.975758,37.3,11.263259,8.213299,4.89412,2.668614,11.159148,116.16375,False,False,False,True,False,1
3,2012.0,10.352891,6.25,3.67772,4.97449,5.612932,43.62,11.574593,8.5035,4.863915,4.401139,10.672236,145.11125,False,False,False,False,False,0
4,2013.0,9.486077,4.641159,0.88295,4.324202,-6.10119,35.855,11.614608,8.724959,3.996117,2.937012,-0.799816,155.77875,False,False,False,True,False,1
5,2014.0,7.188567,2.943686,0.2773,6.05802,6.135386,61.48,11.538279,8.467978,5.27948,3.263981,2.539758,173.9,False,False,False,True,True,2
6,2015.0,9.261046,5.363689,2.91337,5.981092,10.558874,64.5,10.878013,8.013907,5.025942,3.655826,19.203774,208.21875,False,False,False,True,False,1
7,2016.0,10.46055,6.586933,3.57015,5.051767,8.084121,106.1,11.156367,8.296316,5.79411,3.479304,3.345932,217.13375,False,False,False,True,True,2
8,2017.0,10.634328,6.902985,3.79919,4.579376,5.248126,80.08,11.757726,8.965056,5.80646,3.957286,-3.655078,189.1325,False,False,False,False,True,1
9,2018.0,12.329213,7.856213,4.96096,4.50553,4.274084,103.55,12.400207,9.750953,6.48838,3.573476,5.795142,239.0025,False,False,False,True,False,1


In [55]:
lp4 = ld.get_data('IVV', ['TR.FundHoldingRIC','TR.FundHoldingName',
                           'TR.FundPercentageOfFundAssets',
                           'TR.FundNumberOfShares',
                           'TR.FundNumberOfSharesChanged'],
            {'Endnum':'5000'})

In [56]:
lp4

,Instrument,Holding RIC,Holding Name,Percentage of Fund Assets,Number of Shares,Number of Shares Changed
0,IVV,AAPL.OQ,APPLE INC ORD,7.2282,177560243.0,542893.0
1,IVV,NVDA.OQ,NVIDIA CORP ORD,6.059,288145272.0,881056.0
2,IVV,MSFT.OQ,MICROSOFT CORP ORD,5.8361,87334913.0,267020.0
3,IVV,AMZN.OQ,AMAZON.COM INC ORD,3.9281,109929363.0,336141.0
4,IVV,META.OQ,META PLATFORMS INC ORD,2.8803,25607711.0,78278.0
...,...,...,...,...,...,...
506,IVV,SNDK.OQ,SANDISK CORP ORD,0.0,15.0,<NA>
507,IVV,,OTHER FEES,0.0,4.0,0.0
508,IVV,,TOPSIDE ADJUSTMENT,0.0,1.0,0.0
509,IVV,ESH25,EMINI S&P MAR25,-0.0031,4853.0,-877.0
